In [ ]:
%run ../gda/func3.ipynb

In [ ]:
%run ../o_gda/func.ipynb

In [ ]:
def lgda_solver(
    x0: np.ndarray, y0: np.ndarray, w: np.ndarray,
    Ui_L: np.ndarray, Ui_F: np.ndarray, h: np.ndarray,
    p: int, r: int, *,
    eta_x: float = 0.01, eta_y: float = 0.01,
    mu: float = 0.01,
    max_iter: int = 500, tau_interval: int = 10,
    tol: float = 1e-6,
    return_history: bool = False
):
    """LGDA with Mutation (Algorithm 4) solver (final one-shot projection)."""
    x = x0.copy()
    y = y0.copy()
    prev_obj = None

    cp = x.copy()
    cq = y.copy()
    tau = 0

    obj_vals, dx_vals, dy_vals = [], [], []

    for _ in range(max_iter):
        gx = grad_x(x, y, w, Ui_L, Ui_F, h)
        gy = grad_y(x, y, w, Ui_L, Ui_F, h)

        # Mutation-added update steps (no projection here)
        x_tmp = x - eta_x * gx - mu * (x - cp)
        y_tmp = y + eta_y * gy + mu * (cq - y)

        x_next = x_tmp
        y_next = y_tmp

        dx = np.linalg.norm(x_next - x)
        dy = np.linalg.norm(y_next - y)
        new_obj = compute_Lhat(x_next, y_next, w, Ui_L, Ui_F, h)

        obj_vals.append(new_obj)
        dx_vals.append(dx)
        dy_vals.append(dy)

        # 収束判定（tol 使用）
        small_update = max(dx, dy) < tol
        small_obj_change = (prev_obj is not None) and (abs(new_obj - prev_obj) < tol)

        x, y = x_next, y_next
        prev_obj = new_obj

        # if small_update or small_obj_change:
        #     break

        tau += 1
        if tau == tau_interval:
            cp = x.copy()
            cq = y.copy()
            tau = 0

    # ---- 最後の最後に一度だけ射影 ----
    x_final = project_cardinality(x, p)
    y_final = project_cardinality(y, r, mask=(x_final > 0))
    obj_final = compute_Lhat(x_final, y_final, w, Ui_L, Ui_F, h)

    history = {
        "objective": np.array(obj_vals),
        "dx": np.array(dx_vals),
        "dy": np.array(dy_vals)
    }

    if return_history:
        return x_final, y_final, obj_final, history
    return x_final, y_final, obj_final


In [ ]:
def lgda_solver_every_proj(
    x0: np.ndarray, y0: np.ndarray, w: np.ndarray,
    Ui_L: np.ndarray, Ui_F: np.ndarray, h: np.ndarray,
    p: int, r: int, *,
    eta_x: float = 0.01, eta_y: float = 0.01,
    mu: float = 0.01,
    max_iter: int = 500, tau_interval: int = 10,
    tol: float = 1e-6,
    return_history: bool = False
):
    """LGDA with Mutation (Algorithm 4) solver (final one-shot projection)."""
    x = x0.copy()
    y = y0.copy()
    prev_obj = None

    cp = x.copy()
    cq = y.copy()
    tau = 0

    obj_vals, dx_vals, dy_vals = [], [], []

    for _ in range(max_iter):
        gx = grad_x(x, y, w, Ui_L, Ui_F, h)
        gy = grad_y(x, y, w, Ui_L, Ui_F, h)

        # Mutation-added update steps (no projection here)
        x_tmp = x - eta_x * gx - mu * (x - cp)
        y_tmp = y + eta_y * gy + mu * (cq - y)

        #x_next = project_cardinality(x_tmp, p)
        #y_next = project_cardinality(y_tmp, r, mask=(x_next > 0))
        x_next = x_tmp
        y_next = y_tmp

        dx = np.linalg.norm(x_next - x)
        dy = np.linalg.norm(y_next - y)
        new_obj = compute_Lhat(x_next, y_next, w, Ui_L, Ui_F, h)

        obj_vals.append(new_obj)
        dx_vals.append(dx)
        dy_vals.append(dy)

        # 収束判定（tol 使用）
        small_update = max(dx, dy) < tol
        small_obj_change = (prev_obj is not None) and (abs(new_obj - prev_obj) < tol)

        x, y = x_next, y_next
        prev_obj = new_obj

        # if small_update or small_obj_change:
        #     break

        tau += 1
        if tau == tau_interval:
            cp = x.copy()
            cq = y.copy()
            tau = 0

    # ---- 最後の最後に一度だけ射影 ----
    x_final = project_cardinality(x, p)
    y_final = project_cardinality(y, r, mask=(x_final > 0))
    obj_final = compute_Lhat(x_final, y_final, w, Ui_L, Ui_F, h)

    history = {
        "objective": np.array(obj_vals),
        "dx": np.array(dx_vals),
        "dy": np.array(dy_vals)
    }

    if return_history:
        return x_final, y_final, obj_final, history
    return x_final, y_final, obj_final


gif作成

In [ ]:
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os

def create_alpha_beta_gif(x0, y0, wij_matrix, Ui_L, Ui_F, h_i, p, r,
                           fixed_param='alpha',
                           fixed_value=0.01,
                           var_range=np.linspace(0.001, 0.05, 20),
                           filename="alpha_beta_evolution.gif"):
    """
    α または β を固定しつつ、もう一方を変化させたときの解の変化をGIFで可視化する関数。
    """
    temp_dir = "gif_frames"
    os.makedirs(temp_dir, exist_ok=True)
    image_paths = []

    demand_x = [pt[0] for pt in demand_points]
    demand_y = [pt[1] for pt in demand_points]
    candidate_x = [pt[0] for pt in candidate_sites]
    candidate_y = [pt[1] for pt in candidate_sites]

    for idx, var_val in enumerate(var_range):
        eta_x = fixed_value if fixed_param == 'beta' else var_val
        eta_y = fixed_value if fixed_param == 'alpha' else var_val

        x_opt, y_opt, _ = lgda_solver(
            x0, y0, wij_matrix, Ui_L, Ui_F, h_i,
            p, r,
            eta_x=eta_x,
            eta_y=eta_y,
            mu=1.01,
            max_iter=300,
            tau_interval=5,
            return_history=True
        )

        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_xlim(0, 50)
        ax.set_ylim(0, 50)
        ax.set_title(f"{fixed_param} fixed: {fixed_value}, varying: {var_val:.4f}")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.grid(True)

        # Plot base points
        ax.scatter(demand_x, demand_y, color='black', marker='x', label="Demand Points")
        ax.scatter(candidate_x, candidate_y, color='gray', label="Candidate Sites")

        # Plot x_bin
        for i, val in enumerate(x_opt):
            if val == 1:
                ax.scatter(candidate_sites[i][0], candidate_sites[i][1],
                           s=150, facecolors='none', edgecolors='blue', linewidths=2)

        # Plot y_bin
        for i, val in enumerate(y_opt):
            if val == 1:
                ax.scatter(candidate_sites[i][0], candidate_sites[i][1],
                           s=100, color='red')

        ax.legend(loc="best")

        # Save frame
        path = os.path.join(temp_dir, f"frame_{idx:03d}.png")
        plt.savefig(path)
        image_paths.append(path)
        plt.close(fig)

    # Create gif
    images = [imageio.imread(path) for path in image_paths]
    imageio.mimsave(filename, images, duration=0.5)

    # Cleanup
    for path in image_paths:
        os.remove(path)
    os.rmdir(temp_dir)

    print(f"✅ GIF saved as '{filename}'")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os

def create_history_evolution_gif(
    x0, y0, wij_matrix_fn, Ui_L, Ui_F, h_i,
    p, r,
    fixed_param='alpha',  # 'alpha' or 'beta'
    fixed_value=0.01,
    var_range=np.linspace(0.001, 0.05, 20),
    filename="history_gif.gif",
    eta_x=0.01, eta_y=0.01,
    mu=0.01,
    max_iter=300,
    tau_interval=5,
    logy=True
):
    """
    α または β を固定し、もう一方を変化させたときの history（objective, dx, dy）を
    1フレームで可視化し、GIFにする。
    """

    temp_dir = "gif_frames_history"
    os.makedirs(temp_dir, exist_ok=True)
    image_paths = []

    for idx, val in enumerate(var_range):
        if fixed_param == 'alpha':
            alpha = fixed_value
            beta = val
        else:
            alpha = val
            beta = fixed_value

        wij_matrix = wij_matrix_fn(alpha, beta)

        _, _, history = lgda_solver(
            x0, y0, wij_matrix, Ui_L, Ui_F, h_i,
            p, r,
            eta_x=eta_x, eta_y=eta_y,
            mu=mu,
            max_iter=max_iter,
            tau_interval=tau_interval,
            return_history=True
        )

        L_vals = np.asarray(history["objective"])
        dx_vals = np.asarray(history["dx"])
        dy_vals = np.asarray(history["dy"])
        iters = np.arange(1, len(L_vals) + 1)

        fig, axs = plt.subplots(3, 1, figsize=(6, 10))

        axs[0].plot(iters, L_vals, color="tab:blue", linewidth=1.5, label="Objective")
        axs[0].set_title(f"Objective ({fixed_param}={fixed_value:.3f}, Var={val:.3f})")
        axs[0].set_xlabel("Iteration")
        axs[0].set_ylabel("Objective")
        axs[0].grid(True)
        axs[0].legend()
        axs[0].set_ylim(0.3, 1.0)

        axs[1].plot(iters, dx_vals, linestyle="--", color="tab:orange", label="‖dx‖", linewidth=1.5)
        axs[1].set_title("‖dx‖ over Iterations")
        axs[1].set_xlabel("Iteration")
        axs[1].set_ylabel("‖dx‖")
        axs[1].grid(True)
        axs[1].legend()
        axs[1].set_ylim(-1, 4)
        if logy:
            axs[1].set_yscale("log")

        axs[2].plot(iters, dy_vals, linestyle=":", color="tab:green", label="‖dy‖", linewidth=1.5)
        axs[2].set_title("‖dy‖ over Iterations")
        axs[2].set_xlabel("Iteration")
        axs[2].set_ylabel("‖dy‖")
        axs[2].grid(True)
        axs[2].legend()
        axs[2].set_ylim(-1, 4)
        if logy:
            axs[2].set_yscale("log")

        plt.tight_layout()
        frame_path = os.path.join(temp_dir, f"frame_{idx:03d}.png")
        plt.savefig(frame_path)
        image_paths.append(frame_path)
        plt.close(fig)

    # GIF作成
    images = [imageio.imread(path) for path in image_paths]
    imageio.mimsave(filename, images, duration=0.5)

    # クリーンアップ
    for path in image_paths:
        os.remove(path)
    os.rmdir(temp_dir)

    print(f"✅ GIF saved as '{filename}'")


In [ ]:
def create_history_evolution_gif(
    x0, y0, wij_matrix_fn, Ui_L, Ui_F, h_i,
    p, r,
    fixed_param='alpha',
    fixed_value=0.01,
    var_range=np.linspace(0.001, 0.05, 20),
    filename="history_gif.gif",
    eta_x=0.01, eta_y=0.01,
    mu=0.01,
    max_iter=300,
    tau_interval=5,
    logy=True,
    mode="eta_x"  # "eta_x", "eta_y", "mu"
):
    temp_dir = "gif_frames_history"
    os.makedirs(temp_dir, exist_ok=True)
    image_paths = []

    for idx, val in enumerate(var_range):
        alpha = fixed_value if fixed_param == "alpha" else 0.01
        beta = fixed_value if fixed_param == "beta" else 0.01
        wij_matrix = wij_matrix_fn(alpha, beta)

        eta_x_var = val if mode == "eta_x" else eta_x
        eta_y_var = val if mode == "eta_y" else eta_y
        mu_var = val if mode == "mu" else mu

        _, _, history = lgda_solver(
            x0, y0, wij_matrix, Ui_L, Ui_F, h_i,
            p, r,
            eta_x=eta_x_var,
            eta_y=eta_y_var,
            mu=mu_var,
            max_iter=max_iter,
            tau_interval=tau_interval,
            return_history=True
        )

        L_vals = np.asarray(history["objective"])
        dx_vals = np.asarray(history["dx"])
        dy_vals = np.asarray(history["dy"])
        iters = np.arange(1, len(L_vals) + 1)

        fig, axs = plt.subplots(3, 1, figsize=(6, 10))

        axs[0].plot(iters, L_vals, color="tab:blue", linewidth=1.5, label="Objective")
        axs[0].set_title(f"{mode} = {val:.4f}")
        axs[0].set_xlabel("Iteration")
        axs[0].set_ylabel("Objective")
        axs[0].grid(True)
        axs[0].legend()
        axs[0].set_ylim(0.3, 1.0)

        axs[1].plot(iters, dx_vals, linestyle="--", color="tab:orange", label="‖dx‖", linewidth=1.5)
        axs[1].set_title("‖dx‖ over Iterations")
        axs[1].set_xlabel("Iteration")
        axs[1].set_ylabel("‖dx‖")
        axs[1].grid(True)
        axs[1].legend()
        axs[1].set_ylim(-1, 4)
        if logy:
            axs[1].set_yscale("log")

        axs[2].plot(iters, dy_vals, linestyle=":", color="tab:green", label="‖dy‖", linewidth=1.5)
        axs[2].set_title("‖dy‖ over Iterations")
        axs[2].set_xlabel("Iteration")
        axs[2].set_ylabel("‖dy‖")
        axs[2].grid(True)
        axs[2].legend()
        axs[2].set_ylim(-1, 4)
        if logy:
            axs[2].set_yscale("log")

        plt.tight_layout()
        frame_path = os.path.join(temp_dir, f"frame_{idx:03d}.png")
        plt.savefig(frame_path)
        image_paths.append(frame_path)
        plt.close(fig)

    images = [imageio.imread(path) for path in image_paths]
    imageio.mimsave(filename, images, duration=0.5)

    for path in image_paths:
        os.remove(path)
    os.rmdir(temp_dir)

    print(f"✅ GIF saved as '{filename}'")


In [ ]:

def create_growing_sites_gif(
    D, J,
    p, r,
    num_rows_columns = 50,
    filename="growing_sites.gif",
    eta_x=0.01, eta_y=0.01,
    mu=0.01,
    max_iter=300,
    tau_interval=5,
    logy=True,
    step=5
):
    temp_dir = "gif_frames_growing"
    os.makedirs(temp_dir, exist_ok=True)
    image_paths = []

    n_candidates = J
    n_demands = D

    for idx, num in enumerate(range(n_candidates, n_candidates + step*20, step)):
        demand_points, candidate_sites = generate_instance(num_rows_columns, D, J, seed=42)

        distances = compute_distances(demand_points, candidate_sites)
        wij_matrix = compute_wij_matrix(distances)

        Ui_L_sub = compute_Ui_L(wij_matrix, {})
        Ui_F_sub = compute_Ui_F(wij_matrix, {})
        h_i_sub = np.full(D, 1 / D)

        x0_sub = np.random.rand(len(candidate_sites))
        y0_sub = np.random.rand(len(candidate_sites))

        _, _, history = lgda_solver(
            x0_sub, y0_sub,
            wij_matrix,
            Ui_L_sub,
            Ui_F_sub,
            h_i_sub,
            p=min(p, len(candidate_sites)),
            r=min(r, len(candidate_sites)),
            eta_x=eta_x,
            eta_y=eta_y,
            mu=mu,
            max_iter=max_iter,
            tau_interval=tau_interval,
            return_history=True
        )

        L_vals = np.asarray(history["objective"])
        dx_vals = np.asarray(history["dx"])
        dy_vals = np.asarray(history["dy"])
        iters = np.arange(1, len(L_vals) + 1)
        
        print(f'Lval[0]: {L_vals[0]}')

        fig, axs = plt.subplots(3, 1, figsize=(6, 10))

        axs[0].plot(iters, L_vals, color="tab:blue", linewidth=1.5, label="Objective")
        axs[0].set_title(f"Num Sites = {num}")
        axs[0].set_xlabel("Iteration")
        axs[0].set_ylabel("Objective")
        axs[0].grid(True)
        axs[0].legend()
        axs[0].set_ylim(-1.0, 1.0)

        axs[1].plot(iters, dx_vals, linestyle="--", color="tab:orange", label="‖dx‖", linewidth=1.5)
        axs[1].set_title("‖dx‖ over Iterations")
        axs[1].set_xlabel("Iteration")
        axs[1].set_ylabel("‖dx‖")
        axs[1].grid(True)
        axs[1].legend()
        axs[1].set_ylim(-1, 4)
        if logy:
            axs[1].set_yscale("log")

        axs[2].plot(iters, dy_vals, linestyle=":", color="tab:green", label="‖dy‖", linewidth=1.5)
        axs[2].set_title("‖dy‖ over Iterations")
        axs[2].set_xlabel("Iteration")
        axs[2].set_ylabel("‖dy‖")
        axs[2].grid(True)
        axs[2].legend()
        axs[2].set_ylim(-1, 4)
        if logy:
            axs[2].set_yscale("log")

        plt.tight_layout()
        frame_path = os.path.join(temp_dir, f"frame_{idx:03d}.png")
        plt.savefig(frame_path)
        image_paths.append(frame_path)
        plt.close(fig)

    images = [imageio.imread(p) for p in image_paths]
    imageio.mimsave(filename, images, duration=0.8)

    for p in image_paths:
        os.remove(p)
    os.rmdir(temp_dir)

    print(f"✅ GIF saved as '{filename}'")
